In [ ]:
import pandas as pd
import numpy as np
import os
import cloudpickle as pickle

In [ ]:
os.chdir("datasets/tcga/BRCA/mirna_allsamples/")

In [ ]:
df_topics = pd.read_csv("trisbm/trisbm_level_0_topic-dist.csv", index_col=1).drop("i_doc", 1)
df_keywords = pd.read_csv("trisbm/trisbm_level_0_metadatum-dist.csv", index_col=1).drop("i_doc", 1)
df_files = pd.read_csv("files.dat", index_col=0).dropna()

In [ ]:
bins = np.linspace(0,100,20)
avg = df_files["cases.0.diagnoses.0.age_at_diagnosis"].mean(skipna=True)
df_files["age_at_diagnosis"]=pd.cut(df_files["cases.0.diagnoses.0.age_at_diagnosis"].fillna(avg)/365., bins=bins, labels = (bins[1:]+bins[:-1])/2).__array__()

In [ ]:
df_files["gender"] = (df_files["cases.0.demographic.gender"]=="male").astype(int)

In [ ]:
for letter in ["a", "b", "c"]:
    for (old, new) in zip(["stage %s%s"%(i,letter) for i in ["i", "ii", "iii", "iv"]],["stage %s"%i for i in ["i", "ii", "iii", "iv"]]):
        df_files.replace(old, new, inplace=True)
df_files["cases.0.diagnoses.0.tumor_stage"].unique()

In [ ]:
df_files["tumor_stage"]=df_files["cases.0.diagnoses.0.tumor_stage"]
for i,stage in enumerate(["stage i", "stage ii", "stage iii", "stage iv", "stage v", "stage x"]):
    df_files["tumor_stage"].replace(stage, i+1, inplace=True)

In [ ]:
#0 = Alive
df_files["vital_status"]=(df_files["cases.0.demographic.vital_status"]=="Dead").astype(int)

In [ ]:
def get_survival(case):
    if case["cases.0.demographic.vital_status"] == 1:
        return case["cases.0.demographic.days_to_death"]
    else:
        return case["cases.0.diagnoses.0.days_to_last_follow_up"]

df_files["days_survival"] = df_files.apply(get_survival,1)

In [ ]:
df_files = df_files.dropna(subset=["days_survival"])
df_files.set_index("cases.0.submitter_id", inplace=True)

In [ ]:
dim_max = max(df_topics.shape[1], df_keywords.shape[1])

In [ ]:
df_topics = df_topics.reindex(index=df_files.index)
df_keywords = df_keywords.reindex(index=df_files.index)

In [ ]:
X_train = []
Y_train = []

for sample in df_files.index:
    x_1 = df_keywords.loc[sample,:].values
    X_train.append(
    [
        df_topics.loc[sample,:].values,
        np.concatenate([x_1, np.repeat(0, dim_max-len(x_1))])
    ])
    Y_train.append(df_files.at[sample, "days_survival"])

In [ ]:
with open("X_train.pkl","wb") as file:
    pickle.dump(X_train, file, protocol=4)
    
with open("Y_train.pkl","wb") as file:
    pickle.dump(Y_train, file, protocol=4)